In [ ]:
from astroquery.sdss import SDSS
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table 


In [ ]:

def Sloan(pos , bands = ['u','g','r','i','z'] , size = 20):
    """
    This function simplifies the process of SDSS image retrieving.

    Parameters
    ----------
    pos : astropy.coordinates.sky_coordinate.SkyCoord
        position information generated by astropy. It could be either from name or Ra and Dec.
    bands : list, optional
        SDSS filter selection, by default ['u','g','r','i','z']
    size : int, optional
        The searching cone size, by default 20 [arcsec]
    """
    # Query that region
    xid = SDSS.query_region(pos, spectro=False,radius = size * u.arcsec)

    # a to c process the duplicates.
    a = xid.to_pandas()
    b = a.drop_duplicates(subset=['run','rerun','camcol','field'])
    c = Table([b.ra,b.dec,b.objid,b.run,b.rerun,b.camcol,b.field],names = ('ra','dec','objid','run','rerun','camcol','field'))

    # Start downloading
    for band in bands: # for each band
        im = SDSS.get_images(matches=c, band=band)
        i = 0 # Give the images a counter e.g., SDSS_u_0, SDSS_u_1
        for image in im:
            image.writeto(f'SDSS_{band}_{i}.fits', overwrite=True)
            print(f'{band} image downloaded')
            i += 1


In [ ]:
Sloan(SkyCoord.from_name("PTF09gn"))